<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/ask_overlap30_mixed_pubmed_bigtweet_ask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'AskAPatient.fold-0.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-0.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heel pain
2,4,purple sploches around waist
3,7,goiter
4,8,ARTHRITIS


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient_overlap_30.csv",header=None)
test.head()

,0,1
0,0,'scared' feeling
1,3,cracking sensations in my joints
2,4,purple sploches around waist
3,5,fast and slow heartbeat
4,5,increased heart rate


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'mixed_pubmed_bigtweet_askpatient.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'pain',
 'of',
 'the',
 'in',
 'lipitor',
 'and',
 'to',
 'a',
 'muscle',
 'with',
 'for',
 'was',
 'diclofenac',
 'were',
 'severe',
 'on',
 'is',
 'my',
 'myalgia',
 'lower',
 'rt',
 'fatigue',
 'by',
 'loss',
 'weakness',
 'limb',
 'sodium',
 'cramp',
 'disease',
 'at',
 'stomach',
 'that',
 'or',
 'as',
 'back',
 '-',
 'depression',
 'patients',
 'from',
 'this',
 'cramps',
 'not',
 'i',
 'upper',
 'joint',
 'gas',
 'drug',
 'memory',
 'leg',
 'mg',
 'you',
 'pains',
 'all',
 'legs',
 'headache',
 'be',
 'knee',
 'group',
 'are',
 'shoulder',
 'study',
 'unable',
 'neck',
 'gastrointestinal',
 'arthralgia',
 'treatment',
 'excessive',
 'foot',
 'it',
 'arthritis',
 'aches',
 'after',
 'hip',
 'an',
 'amp',
 'have',
 'numbness',
 'voltaren',
 'like',
 'arthrotec',
 'release',
 'insomnia',
 'p',
 'no',
 '/',
 'both',
 'increased',
 'nausea',
 'bleeding',
 'gluten',
 'time',
 'asthenia',
 'abdom

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.849952,3.505333,0.438799,34:02


In [22]:
# learn.fit_one_cycle(1, 1e-2)

In [23]:
# learn.fit_one_cycle(1, 1e-3)

In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.526260,3.443261,0.444527,34:10


In [25]:
learn.save_encoder('pubmed_bigtweet_askpatient_mixed_overlap30_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_bigtweet_askpatient_mixed_overlap30_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.822152,3.499183,0.397633,01:00


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.918651,3.075610,0.471006,01:22


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.376613,2.614435,0.539645,03:23


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.907655,2.293214,0.603550,03:17
1,1.679577,2.163075,0.628402,03:05


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.527790,1.963409,0.674556,03:24
1,1.299670,1.872911,0.685207,03:21


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.191469,1.759127,0.688757,03:17
1,1.012636,1.692991,0.707692,03:19


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.968581,1.614565,0.714793,03:20
1,0.882861,1.547110,0.725444,03:16


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.813533,1.519891,0.733728,03:23
1,0.705939,1.483210,0.747929,03:20


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.668407,1.432823,0.756213,03:27
1,0.651724,1.430973,0.759763,03:07
2,0.620419,1.357590,0.766864,03:21
3,0.495676,1.398758,0.758580,03:21


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.500461,1.407939,0.757396,03:04
1,0.526735,1.333597,0.770414,03:27
2,0.403958,1.308362,0.785799,03:11
3,0.387642,1.281552,0.785799,03:20


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.385561,1.360166,0.781065,03:19
1,0.393209,1.372682,0.788166,03:13
2,0.340875,1.308661,0.789349,03:11
3,0.283664,1.315560,0.791716,03:05


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.292784,1.395457,0.788166,03:03
1,0.368823,1.317369,0.785799,03:05
2,0.291471,1.330873,0.791716,03:12
3,0.268578,1.304128,0.794083,03:11


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.271724,1.354083,0.792899,03:15
1,0.300611,1.328905,0.795266,03:20
2,0.261318,1.303908,0.796450,03:15
3,0.229796,1.339092,0.800000,03:30


In [39]:
# learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

In [40]:
# learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

In [41]:
# learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

In [42]:
# learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.253098,1.345344,0.798817,03:30


In [44]:
# save the best model

learn.save_encoder('pubmed_bigtweet_askpatient_mixed_overlap30')

# Part three: Predict on the test dataset

In [45]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [46]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

872
872
[166, 876, 438, 334, 5, 530, 1026, 8, 8, 8, 8, 760, 98, 520, 5, 19, 21, 135, 27, 644, 644, 29, 289, 289, 211, 31, 32, 34, 289, 37, 218, 623, 469, 255, 99, 99, 49, 94, 49, 49, 49, 49, 100, 100, 100, 100, 891, 53, 61, 70, 71, 71, 71, 76, 76, 289, 228, 614, 591, 628, 79, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 182, 289, 591, 79, 79, 80, 80, 80, 80, 615, 88, 89, 137, 391, 591, 91, 84, 94, 94, 94, 94, 94, 96, 98, 598, 98, 615, 260, 100, 265, 100, 100, 100, 416, 49, 291, 236, 34, 108, 108, 209, 879, 409, 111, 130, 4, 111, 281, 717, 182, 565, 65, 120, 1009, 298, 125, 125, 125, 125, 125, 940, 125, 127, 741, 289, 289, 289, 312, 627, 142, 144, 479, 794, 151, 30, 157, 157, 509, 165, 166, 166, 166, 291, 194, 172, 614, 172, 172, 172, 172, 180, 182, 182, 182, 182, 182, 182, 183, 183, 183, 183, 183, 184, 185, 187, 187, 187, 757, 189, 189, 190, 190, 190, 190, 190, 190, 190, 190, 190, 193, 197, 34, 34, 34, 34, 736, 205, 769, 210, 210, 210, 210, 210, 211, 717, 

In [47]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

485
0.5561926605504587
